In [1]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import math

# Task 1

In [163]:
# Callback function for the mouse
positions=[] 
count=0
def draw_circle(event,x,y,flags,param):
    # If the event is a Left Button Click, the coordinates should be saved in the lists.
    global positions,count
    if event == cv2.EVENT_LBUTTONUP:
        cv2.circle(stereo,(x,y),2,(255,0,0),-1)
        positions.append([x,y])
        count+=1


# Reading the image and storing it in variable stereo
stereo = cv2.imread('Python user/stereo2012a.png')

# window defining
cv2.namedWindow('image')

cv2.setMouseCallback('image',draw_circle)

while(True):
    cv2.imshow('image',stereo)
    k = cv2.waitKey(20) & 0xFF
    # Exits when pressed Escape or when all 6 points are selected
    if k == 27 or count==6:
        break

cv2.destroyAllWindows()


In [164]:
cv2.imshow('Stereo with chosen xyz points', stereo)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 

In [25]:
uv=positions.copy()
print(uv)

[[358, 299], [402, 308], [376, 380], [333, 368], [284, 312], [275, 264]]


In [140]:
def Normalization(nd, x):
    '''
    Coordinate normalisation (centroid to origin and sqrt mean distance) (2 or 3).
    Input
    -----
    nd: number of dimensions, 3 in our case
    x: the data that needs to be normalised (various directions in various columns and points in various rows)
    Output
    ------
    Tr: the matrix of transformations (translation plus scaling)
    x: the data that has been altered
    '''

    x = np.asarray(x)
    m, s = np.mean(x, 0), np.std(x)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x


def calibrate(nd, xyz, uv):
    '''
    %% TASK 1: CALIBRATE
    %
    % Function to perform camera calibration
    %
    % Usage:   calibrate(image, XYZ, uv)
    %          return C
    %   Where:   image - is the image of the calibration target.
    %            XYZ - is a N x 3 array of  XYZ coordinates
    %                  of the calibration target points. 
    %            uv  - is a N x 2 array of the image coordinates
    %                  of the calibration target points.
    %            K   - is the 3 x 4 camera calibration matrix.
    %  The variable N should be an integer greater than or equal to 6.
    %
    %  This function plots the uv coordinates onto the image of the calibration
    %  target. 
    %
    %  It also projects the XYZ coordinates back into image coordinates using
    %  the calibration matrix and plots these points too as 
    %  a visual check on the accuracy of the calibration process.
    %
    %  Lines from the origin to the vanishing points in the X, Y and Z
    %  directions are overlaid on the image. 
    %
    %  The mean squared error between the positions of the uv coordinates 
    %  and the projected XYZ coordinates is also reported.
    %
    %  The function should also report the error in satisfying the 
    %  camera calibration matrix constraints.
    % 
    % Saswat Panda, 30/05/2021 
    '''
    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    uv = np.asarray(uv)

    n = xyz.shape[0]

    # Validating the parameters:
    if uv.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, uv.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('The number of coordinates is incorrect (%d) for %dD DLT (It ought to be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires a minimum of %d calibration points. There were only %d points chosen.' %(nd, 2*nd, n))
        
    # To improve the DLT quality, normalise the data (Since, The system of coordinates has an impact on DLT.).
    # When there is a significant perspective distortion, this is important.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Normalization(nd, xyz)
    Tuv, uvn = Normalization(2, uv)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = uvn[i, 0], uvn[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Converting A to numpy array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # In the last line of Vh the parameters should be normalized
    L = V[-1, :] / V[-1, -1]
    # Camera projection matrix
    H = L.reshape(3, nd + 1)

    # Denormalizing
    # pinv is the Moore-Penrose pseudo-inverse of a matrix, which is the generalised inverse of 
    # a matrix, calculated using its SVD.
    H = np.dot( np.dot( np.linalg.pinv(Tuv), H ), Txyz )

    H = H / H[-1, -1]
    L = L.reshape((3,4))


    # The DLT's mean error (In units of camera coordinates, the mean residual of the DLT transformation) is:
    uv2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    uv2 = uv2 / uv2[2, :] 
    # The mean distance is calculated as below
    err = np.sqrt( np.mean(np.sum( (uv2[0:2, :].T - uv)**2, 1)) ) 

    return L, err



# Selected 3D co-ordinates, set of 6
xyz = [[7,7,0], [14,7,0], [14,0,7], [7,0,7], [0,7,7], [0,14,7]]
# uv found using cv2 for the corresponding xyz co-ordinates
uv = [[358, 299], [402, 308], [376, 380], [333, 368], [284, 312], [275, 264]]

nd = 3
C, err = calibrate(nd, xyz, uv)
print('Calibration Matrix(C)')
print(C)
print('\n DLT Mean Error')
print(err)


Calibration Matrix(C)
[[ 0.66089574 -0.11542858 -0.51452179 -0.00853066]
 [ 0.15304782 -0.7352157   0.35986903 -0.00180281]
 [-0.02592132 -0.02144844 -0.02461616  1.        ]]

 DLT Mean Error
0.1028974043451584


In [141]:
def vgg_rq(S):
    S = S.T
    [Q,U] = np.linalg.qr(S[::-1,::-1], mode='complete')

    Q = Q.T
    Q = Q[::-1, ::-1]
    U = U.T
    U = U[::-1, ::-1]
    if np.linalg.det(Q)<0:
        U[:,0] = -U[:,0]
        Q[0,:] = -Q[0,:]
    return U,Q


def vgg_KR_from_P(P, noscale = True):
    N = P.shape[0]
    H = P[:,0:N]
    print(N,'|', H)
    [K,R] = vgg_rq(H)
    if noscale:
        K = K / K[N-1,N-1]
        if K[0,0] < 0:
            D = np.diag([-1, -1, np.ones([1,N-2])]);
            K = K @ D
            R = D @ R
        
            test = K*R; 
            assert (test/test[0,0] - H/H[0,0]).all() <= 1e-07
    
    t = np.linalg.inv(-P[:,0:N]) @ P[:,-1]
    return K, R, t

K, R, t = vgg_KR_from_P(C)

print('K obtained from C')
print(K)
print('R obtained from C')
print(R)
print('t obtained from C')
print(t)

3 | [[ 0.66089574 -0.11542858 -0.51452179]
 [ 0.15304782 -0.7352157   0.35986903]
 [-0.02592132 -0.02144844 -0.02461616]]
K obtained from C
[[20.24836923  0.12209739 -1.14504132]
 [ 0.         19.90373409  1.6936681 ]
 [ 0.          0.          1.        ]]
R obtained from C
[[ 0.74634901 -0.16076031 -0.64584772]
 [ 0.23736035 -0.84228809  0.48395334]
 [-0.62179033 -0.51449673 -0.59048277]]
t obtained from C
[14.38620037 10.83888278 16.0306739 ]


In [162]:
stereo = cv2.imread('Python user/stereo2012a.png')

x,y,z, one = np.hsplit(np.array([[7,7,0,1], [14,7,0,1], [14,0,7,1], [7,0,7,1], [0,7,7,1], [0,14,7,1]]),4)
xyz_np = np.concatenate([x.reshape(1,6),y.reshape(1,6),z.reshape(1,6),one.reshape(1,6)])

a = np.matmul(K,np.concatenate((R,t.reshape(3,1)),axis=1))

print()
uv_found = np.matmul(a,xyz_np)
u_f, v_f, _ = np.vsplit(uv_found, 3)
# u_f[0][1]
for i in range(u_f.size):
    cv2.circle(stereo,(int(u_f[0][i]),int(v_f[0][i])),2,(255,0,0),-1)
    
cv2.imshow('Projection of XYZ back using Calibration Matrix', stereo)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 


In [31]:
stereo1 = cv2.imread('Python user/stereo2012a.png')
H, W, _ = stereo1.shape
dim = (int(W/2), int(H/2))
stereo1_r = cv2.resize(stereo1, dim, interpolation = cv2.INTER_NEAREST)
cv2.imshow('Stereo with chosen xyz points', stereo)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 

In [32]:
# Callback function for the mouse
positions=[] 
count=0
def draw_circle(event,x,y,flags,param):
    # If the event is a Left Button Click, the coordinates should be saved in the lists.
    global positions,count
    if event == cv2.EVENT_LBUTTONUP:
        cv2.circle(stereo1_r,(x,y),2,(255,0,0),-1)
        positions.append([x,y])
        count+=1


# window defining
cv2.namedWindow('image')

cv2.setMouseCallback('image',draw_circle)

while(True):
    cv2.imshow('image',stereo1_r)
    k = cv2.waitKey(20) & 0xFF
    if k == 27 or count==6:
        break

cv2.destroyAllWindows()

In [33]:
cv2.imshow('Stereo of resized img with xyz points', stereo1_r)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 

In [46]:
uv_rez = positions.copy()
print(uv_rez)

[[179, 149], [201, 154], [188, 190], [167, 184], [142, 156], [137, 132]]


In [127]:
# Selected 3D co-ordinates, set of 6
xyz = [[7,7,0], [14,7,0], [14,0,7], [7,0,7], [0,7,7], [0,14,7]]

# uv_rez = [[179, 149], [201, 154], [188, 190], [167, 184], [142, 156], [137, 132]] 
# Above is the original uv resized list, it has been modified slightly to make it exactly half 
uv_rez = [[179, 149.5], [201, 154], [188, 190], [166.5, 184], [142, 156], [137.5, 132]]
nd = 3
C, err = calibrate(nd, xyz, uv_rez)
print('Calibration Matrix(C\') of resized image')
print(C)
print('\n DLT Mean Error of resized image')
print(err)

Calibration Matrix(C') of resized image
[[ 0.66089574 -0.11542858 -0.51452179 -0.00853066]
 [ 0.15304782 -0.7352157   0.35986903 -0.00180281]
 [-0.02592132 -0.02144844 -0.02461616  1.        ]]

 DLT Mean Error of resized image
0.051448702172584924


In [128]:
K, R, t = vgg_KR_from_P(C)

print('K\' obtained from C')
print(K)
print('R\' obtained from C')
print(R)
print('t\' obtained from C')
print(t)


3 | [[ 0.66089574 -0.11542858 -0.51452179]
 [ 0.15304782 -0.7352157   0.35986903]
 [-0.02592132 -0.02144844 -0.02461616]]
K' obtained from C
[[20.24836923  0.12209739 -1.14504132]
 [ 0.         19.90373409  1.6936681 ]
 [ 0.          0.          1.        ]]
R' obtained from C
[[ 0.74634901 -0.16076031 -0.64584772]
 [ 0.23736035 -0.84228809  0.48395334]
 [-0.62179033 -0.51449673 -0.59048277]]
t' obtained from C
[14.38620037 10.83888278 16.0306739 ]


# Task 2

In [90]:
# Callback function for the mouse
points = []
count=0
def draw_circle(event,x,y,flags,param):
    # If the event is a Left Button Click, the coordinates should be saved in the lists.
    global points,count
    if event == cv2.EVENT_LBUTTONUP:
        cv2.circle(L_building,(x,y),2,(255,0,0),-1)
        points.append([x,y])
        count+=1


# Reading the image and storing it in variable L_building
L_building = cv2.imread('Python user/Left.jpg')

# window defining
cv2.namedWindow('image1')

cv2.setMouseCallback('image1',draw_circle)

while(True):
    cv2.imshow('image1',L_building)
    k = cv2.waitKey(20) & 0xFF
    if k == 27 or count == 4:
        break

cv2.destroyAllWindows()


In [167]:
print(points)

[[175, 152], [177, 175], [235, 185], [234, 159]]


In [ ]:
cv2.imshow('Selected points in Left Building', L_building)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 

In [93]:
# Callback function for the mouse
points1 = []
count=0
def draw_circle(event,x,y,flags,param):
    # If the event is a Left Button Click, the coordinates should be saved in the lists.
    global points1,count
    if event == cv2.EVENT_LBUTTONUP:
        cv2.circle(R_building,(x,y),2,(255,0,0),-1)
        points1.append([x,y])
        count+=1


# Reading the image and storing it in variable R_building
R_building = cv2.imread('Python user/Right.jpg')

# window defining
cv2.namedWindow('image2')

cv2.setMouseCallback('image2',draw_circle)

while(True):
    cv2.imshow('image2',R_building)
    k = cv2.waitKey(20) & 0xFF
    if k == 27 or count == 4:
        break

cv2.destroyAllWindows()

In [74]:
print(points1)

[[208, 184], [209, 205], [284, 204], [284, 182]]


In [95]:
cv2.imshow('Corresponding points in Right Building', R_building)
  
# waits for a key to be pressed by the user
# This is required to prevent the Python kernel from crashing.
cv2.waitKey(0) 
  
# all open windows should be closed
cv2.destroyAllWindows() 

In [165]:
points = np.array([[175, 152], [177, 175], [235, 185], [234, 159]])
points1 = np.array([[208, 184], [209, 205], [284, 204], [284, 182]])
def homography(u2Trans, v2Trans, uBase, vBase):
    '''
    %% TASK 2: 
    % Computes the homography H applying the Direct Linear Transformation 
    % The transformation is such that 
    % p = np.matmul(H, p.T), i.e.,
    % (uBase, vBase, 1).T = np.matmul(H, (u2Trans , v2Trans, 1).T)
    % Note: we assume (a, b, c) => np.concatenate((a, b, c), axis), be careful when 
    % deal the value of axis 
    %
    % INPUTS: 
    % u2Trans, v2Trans - vectors with coordinates u and v of the transformed image point (p') 
    % uBase, vBase - vectors with coordinates u and v of the original base image point p  
    % 
    % OUTPUT 
    % H - a 3x3 Homography matrix  
    % 
    % Saswat Panda, 30/05/2021 
    '''
    global points, points1
    points = np.column_stack((points,np.ones(points.shape[0]))).T
    points1 = np.column_stack((points1,np.ones(points1.shape[0]))).T

    p1 = points[:-1,:].T
    p2 = points1[:-1,:].T



    # Computing the 0th row of A 
    A_up = np.column_stack((p1,np.ones(p1.shape[0]),np.zeros((p1.shape[0],3)),-p1[:,0]*p2[:,0],-p1[:,1]*p2[:,0],-p2[:,0]))
    # Computing the 1st row of A 
    A_below = np.column_stack((np.zeros((p1.shape[0],3)),p1,np.ones(p1.shape[0]),-p1[:,0]*p2[:,1],-p1[:,1]*p2[:,1],-p2[:,1]))

    # Stacking oth and 1st rows
    A = np.vstack((A_up,A_below))

    # Computing the SVD
    result = np.linalg.svd(A)[-1][-1]
    # The parameters should be normalised for the last line too.
    result = result/result[-1]

    # Reshaping the result
    result = result.reshape((p1.shape[1]+1,-1))

    H = result 

    return H 


# horizontally splitting the points and points1 for converting to desired input
H = homography(np.hsplit(points1, 2)[0],np.hsplit(points1, 2)[1],np.hsplit(points, 2)[0],np.hsplit(points, 2)[1])
print("The homography matrix(H) calculated using the DLT algorithm is:")
print(H)

The homography matrix(H) calculated using the DLT algorithm is:
[[ 2.20267020e+00 -2.04532968e-01 -9.03400798e+01]
 [ 1.77345385e-01  1.07672588e+00  3.88744727e+01]
 [ 1.97181842e-03 -4.97722561e-04  1.00000000e+00]]


In [86]:
L_building_1 = cv2.imread('Python user/Left.jpg')
height, width, _ = L_building_1.shape
im1Reg = cv2.warpPerspective(L_building_1, H, (width, height))
# Using cv2.imshow() method 
# Displaying the image 
cv2.imshow('Left warped', im1Reg)
  
#waits for user to press any key 
#(this is necessary to avoid Python kernel form crashing)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows() 

In [88]:
# getting the points for warped image
# Callback function for the mouse
points_w = []
count=0
def draw_circle(event,x,y,flags,param):
    # If the event is a Left Button Click, the coordinates should be saved in the lists.
    global points_w,count
    if event == cv2.EVENT_LBUTTONUP:
        cv2.circle(im1Reg,(x,y),2,(255,0,0),-1)
        points_w.append([x,y])
        count+=1



# window defining
cv2.namedWindow('Points in warped image')

cv2.setMouseCallback('Points in warped image',draw_circle)

while(True):
    cv2.imshow('Points in warped image',im1Reg)
    k = cv2.waitKey(20) & 0xFF
    if k == 27 or count == 4:
        break

cv2.destroyAllWindows()


In [84]:
print(points_w)

[[208, 184], [207, 205], [284, 205], [285, 182]]


In [179]:
points1 = np.array([[208, 184], [209, 205], [284, 204], [284, 182]])
points_w = np.array([[208, 184], [207, 205], [284, 205], [285, 182]])
d = []

for i in range(points1.shape[0]):
    d.append(math.dist(points1[i], points_w[i]))

print('The distance between invidual corresponding points are ',d)

print('The mean of distances ',np.mean(np.array(d)))

The distance between invidual corresponding points are  [0.0, 2.0, 1.0, 1.0]
The mean of distances  1.0
